In [2]:
import pandas as pd

In [3]:
import os

Address_df = pd.read_csv('testdata/address.csv')
ContactPoint_df = pd.read_csv('testdata/ContactPoint.csv')
HealthcareOrganization_df = pd.read_csv('testdata/HealthcareOrganization.csv')
HealthcarePersonnel_df = pd.read_csv('testdata/HealthcarePersonnel.csv')
Person_df = pd.read_csv('testdata/Person.csv')
ServiceDepartment_df = pd.read_csv('testdata/ServiceDepartment.csv')

# Specify the encoding to handle the decoding error
golden_record = pd.read_csv('golden_standard_duplicates.csv', encoding='latin1')

In [20]:


contact_point = ContactPoint_df[ContactPoint_df["identifier"] == "23a7711a-8133-4876-b7eb-dcd9e87a1613"]
language = contact_point['availableLanguage']
language = language.str.strip('[]').str.split(',').str[0]
print(language)

language = language.iloc[0].strip(' ')
print(language)



0    'de'
Name: availableLanguage, dtype: object
'de'


In [3]:
golden_record

,original_id,duplicate_id,entity_type,variation_type,field_name,original_value,varied_value
0,49b564fb-92a6-41d7-8069-c54224039721,3a72a669-e03c-4361-a14e-10ee230c2ebd,Address,city_typo,city,Trieben,Triebben
1,9b3a0c89-1a32-4148-9bba-c83856e9b78d,4db1762e-c9d9-4b15-befa-4e9942045f8c,Address,city_typo,city,Holwierde,Holwiere
2,d4713d60-c8a7-4639-ab11-67b367a9c378,4574a37e-9dbf-493f-bc03-1029253053ae,Address,postal_format,postalCode,6838,6838
3,fa83ada4-a212-4ac5-b689-a4a5ffda0336,e5f14395-22e7-4e59-9414-7ca83455f1f6,Address,country_expansion,country,AT,Austria
4,b732d46f-21e1-4094-9efe-e464da90f534,f95c46f7-682e-4002-a9f1-5f7cd7ea0686,Address,house_number_suffix,text,Ott Roads 92,Ott Roads 92B
...,...,...,...,...,...,...,...
78,a747795e-b0c9-42af-9db3-1510675d0855,49ace049-13bd-41ae-b0dd-4704248988cf,ContactPoint,email_domain_change,email,Ali.Hematologic@dept.healthcare.org,Ali.Hematologic@dept.healthcare.de
79,35475c5e-f76d-4e6e-8726-d44a215203c7,86d6c424-27b2-4a70-a945-2a96a1239c72,ContactPoint,email_typo,email,West.Public@dept.healthcare.org,West.Puhblic@dept.healthcare.org
80,d22a3a62-1109-4978-93a0-81a6a7647989,7d416588-9a34-4cda-9481-4f7f3022b8f1,ContactPoint,email_typo,email,Peterson.Pulmonary@dept.healthcare.org,Peterson.Plumonary@dept.healthcare.org
81,75fd3537-6f1b-4d07-9aff-71ae30f2c485,260aeef2-95dd-44bd-a7c4-bf8d59949066,ContactPoint,email_domain_change,email,Bryan.Cardiovascular@dept.healthcare.org,Bryan.Cardiovascular@dept.healthcare.de


In [4]:
# Dictionary to store all dataframes by entity type
entity_dataframes = {}

# Get a static list of variable names first to avoid the iteration error
variable_names = list(locals().keys())

# Populate the dictionary with your existing dataframes
for var_name in variable_names:
    if var_name.endswith('_df'):
        entity_type = var_name.replace('_df', '')
        entity_dataframes[entity_type] = locals()[var_name]

# Iterate through each row in the golden record
for _, row in golden_record.iterrows():
    entity_type = row['entity_type']
    
    # Check if we have a dataframe for this entity type
    if entity_type in entity_dataframes:
        df = entity_dataframes[entity_type]
        
        # Find the matching row using original_id
        mask = df['identifier'] == row['original_id']
        
        # If a match is found, update the specified field with the varied value
        if mask.any():
            field_name = row['field_name']
            if field_name in df.columns:
                df.loc[mask, field_name] = row['varied_value']
                print(f"Updated {entity_type} - {row['original_id']} - {field_name}: {row['varied_value']}")
            else:
                print(f"Warning: Field {field_name} not found in {entity_type} dataframe")
        else:
            print(f"Warning: No row with identifier {row['original_id']} found in {entity_type} dataframe")
    else:
        print(f"Warning: No dataframe found for entity type {entity_type}")

Updated Address - 49b564fb-92a6-41d7-8069-c54224039721 - city: Triebben
Updated Address - 9b3a0c89-1a32-4148-9bba-c83856e9b78d - city: Holwiere
Updated Address - d4713d60-c8a7-4639-ab11-67b367a9c378 - postalCode: 6838 
Updated Address - fa83ada4-a212-4ac5-b689-a4a5ffda0336 - country: Austria
Updated Address - b732d46f-21e1-4094-9efe-e464da90f534 - text: Ott Roads 92B
Updated Address - f20e8bda-39cc-4d88-832f-594d0b82b61c - text: Aydenhof 120B
Updated Address - ab12fb53-8f42-4ebe-a3b8-70f377cb1a27 - postalCode: 8857 
Updated Address - f172840f-3e19-40ba-ae09-ccfba3dee81f - city: Fiischamend
Updated Address - cb18d6a4-15db-4487-a038-03f32b2bcfda - city: Hellevoetsluuis
Updated HealthcareOrganization - d675ebf7-4fe3-4c9a-9371-0f577e9cf84f - healthcareOrganizationName: HughesP-erry Tervisekeskus
Updated HealthcareOrganization - ae55cdff-34ab-48fd-8a68-e88e0ad40415 - healthcareOrganizationName: MS Gesundheitszentrum
Updated ServiceDepartment - 2347b24f-a0f9-4074-9177-b6d45f2e9167 - serviceD

In [5]:
HealthcareOrganization_df

,identifier,healthcareOrganizationName,address,contactPoint
0,3e70f16a-5548-4822-9e1b-372ad3fbf47a,Faulkner-Howard Gesundheitszentrum,d4713d60-c8a7-4639-ab11-67b367a9c378,23a7711a-8133-4876-b7eb-dcd9e87a1613
1,b306d1a8-e5ee-4c76-948b-2758d7ab7928,"West, Miller and Flores Gesundheitszentrum",247a8333-f7b0-47d2-8da8-056c3d15eef7,1beb3711-7d41-4602-aece-328bff7b118e
2,9b38fe80-3042-4325-a28f-5ab01fdb8b32,"Bryan, Barnes and Hill Gesundheitszentrum",adf20806-e521-4606-b717-6e84d977e993,b341facd-ff0a-40f1-a425-799aa905d750
3,e786ab37-5bca-47be-8298-17c53308fb2e,Robbins Group Zorg,31d0b664-0589-4877-9b02-52440950fd13,efdd35f8-0fa3-4266-8cfd-ba9bba26d851
4,7b59051b-f400-48d7-831d-5a973d792fa1,Peterson LLC Zorg,d86dbf11-2880-4c5d-ad1b-8f60c9e4dab2,aa6524ab-713b-4e05-abe2-136898c75205
5,d675ebf7-4fe3-4c9a-9371-0f577e9cf84f,HughesP-erry Tervisekeskus,b732d46f-21e1-4094-9efe-e464da90f534,313b32b7-9836-4189-8063-e42f14aa451c
6,ae55cdff-34ab-48fd-8a68-e88e0ad40415,MS Gesundheitszentrum,fa83ada4-a212-4ac5-b689-a4a5ffda0336,6ae04d52-adb3-48cb-b315-8c0c66dd7794
7,bb0378eb-7a62-422e-9d69-d9fc4b1cb8bd,"Ali, Lawson and Simpson Gesundheitszentrum",d8ab0b30-0ac0-4f0d-9974-c146e8ec01b3,6f790959-a3e0-4b3b-b56b-0715e7180322
8,8e751eb7-64d0-4913-991b-8adf0202861c,Evans-Rodriguez Zorg,673617d9-4d7b-4307-9224-11e6ba8982dd,564ae909-7958-4e69-bb2e-1b82e89dc815
9,26bdd974-d3b5-44b0-8be0-4c3e5c949381,Richardson LLC Zorg,6c4c3935-379d-4da1-ade6-c5e9b6e355f6,31234efe-6e64-4043-aaa5-0f4ec6f00933


In [6]:
from faker import Faker

fake = Faker()

# Create a copy to avoid modifying original data
A_copy = Address_df.copy()
CP_copy = ContactPoint_df.copy()
P_copy = Person_df.copy()
HO_copy = HealthcareOrganization_df.copy()
SD_copy = ServiceDepartment_df.copy()
HP_copy = HealthcarePersonnel_df.copy()

# Save old values
old_valuesHO = {
    'identifier': HO_copy['identifier'].copy(),
    'address': HO_copy['address'].copy(),
    'contactPoint': HO_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field
HO_copy['identifier'] = HO_copy['identifier'].apply(lambda x: fake.uuid4())
HO_copy['address'] = HO_copy['address'].apply(lambda x: fake.uuid4())
HO_copy['contactPoint'] = HO_copy['contactPoint'].apply(lambda x: fake.uuid4())

# Create copies of dataframes
old_valuesSD = {
    'identifier': SD_copy['identifier'].copy(),
    'address': SD_copy['address'].copy(),
    'contactPoint': SD_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field in ServiceDepartment_df
SD_copy['identifier'] = SD_copy['identifier'].apply(lambda x: fake.uuid4())
SD_copy['address'] = SD_copy['address'].apply(lambda x: fake.uuid4())
SD_copy['contactPoint'] = SD_copy['contactPoint'].apply(lambda x: fake.uuid4())

old_valuesHP = {
    'identifier': HP_copy['identifier'].copy()
}
# Generate new UUIDs for Healthcare Personnel identifiers
HP_copy['identifier'] = HP_copy['identifier'].apply(lambda x: fake.uuid4())

In [7]:
HO_copy

,identifier,healthcareOrganizationName,address,contactPoint
0,8a0f6258-8687-42d8-9a76-a6220d477718,Faulkner-Howard Gesundheitszentrum,2e9a8e08-7f8c-430f-b312-70d398757209,ed56848e-465a-4f45-828d-78055d3dc49f
1,0ed2f52b-b48b-4572-87a7-ac699ea1f6d2,"West, Miller and Flores Gesundheitszentrum",c69c5caa-c1a9-4dee-98ed-14df1cc92562,274dc81a-aba0-460b-87d3-e7257ae86d83
2,055ae6de-6ef0-429d-b7b5-4f0a08c87898,"Bryan, Barnes and Hill Gesundheitszentrum",ad1c95fc-03f8-415c-b8d8-8433efb802a8,e6d95a8e-01c5-4534-8ee1-9556ab3a5875
3,76ec2eb6-7844-4ac0-8d63-cfa1b4016e74,Robbins Group Zorg,02feac9a-aebb-4692-b177-9eb66582fd29,f45299d3-466d-408c-aeaf-4a06fdf5bfaa
4,c79b4739-161b-400c-8109-15d390489d3c,Peterson LLC Zorg,250713a3-ce19-48f8-a21e-4e55c3244ef9,7bb499ce-8ff3-4f2f-91d5-ab9b45d0d440
5,9a396f78-73c3-42ff-8a86-f9e50490b88f,HughesP-erry Tervisekeskus,62f6170a-2e0f-41c8-9cb6-fe64574fcddf,e46bfd2f-6463-47b9-9a35-b0fd41b611c5
6,e10f5aca-4c5f-4d77-bbd5-5c5b00c5dc64,MS Gesundheitszentrum,0f4ea7b5-ecd5-47ee-a383-9d0313768297,2736043e-bcfb-4ebb-ab47-4af30267c810
7,08527049-e432-472a-8b84-a1dda528dd88,"Ali, Lawson and Simpson Gesundheitszentrum",e2f04435-1c74-453e-bfcd-465552faddc0,29f5285b-3b01-479b-85a4-fc5b384e644b
8,aa185193-367e-482b-a0a0-d523acfb8d65,Evans-Rodriguez Zorg,fbe81007-2d11-411d-9ff2-6b3a9d527c3a,755df120-e0e6-426c-aa30-1d8c621052fe
9,47ad62a2-7adb-4637-9218-42a19303b9e4,Richardson LLC Zorg,bd7e62a7-e70c-4ce6-b833-cf65dc65dd92,42c4bba1-5772-4a23-ae38-493f5b6202e8


In [8]:
# Create a mapping of old to new UUIDs
HO_uuid_mapping = {
    old_valuesHO['identifier'].iloc[i]: HO_copy['identifier'].iloc[i]
    for i in range(len(HO_copy))
}

# Update references in other dataframes (using the copies)
# Update institution references in Healthcare_Personnel_df copy
HP_copy['institution'] = HP_copy['institution'].map(HO_uuid_mapping).fillna(HP_copy['institution'])

# Update isPartOf references in ServiceDepartment_df copy
SD_copy['isPartOf'] = SD_copy['isPartOf'].map(HO_uuid_mapping).fillna(SD_copy['isPartOf'])

# Update contact point references
ho_contact_mapping = {
    old_valuesHO['contactPoint'].iloc[i]: HO_copy['contactPoint'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in ContactPoint_df copy where it matches old contact points
CP_copy['identifier'] = CP_copy['identifier'].map(ho_contact_mapping).fillna(CP_copy['identifier'])

# Update address references
ho_address_mapping = {
    old_valuesHO['address'].iloc[i]: HO_copy['address'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in Address_df copy where it matches old addresses
A_copy['identifier'] = A_copy['identifier'].map(ho_address_mapping).fillna(A_copy['identifier'])

# Create mapping for ServiceDepartment
sd_uuid_mapping = {
    old_valuesSD['identifier'].iloc[i]: SD_copy['identifier'].iloc[i]
    for i in range(len(SD_copy))
}

# Create address mapping for ServiceDepartment
sd_address_mapping = {
    old_valuesSD['address'].iloc[i]: SD_copy['address'].iloc[i]
    for i in range(len(SD_copy))
}

# Create contact point mapping for ServiceDepartment
sd_contact_mapping = {
    old_valuesSD['contactPoint'].iloc[i]: SD_copy['contactPoint'].iloc[i]
    for i in range(len(SD_copy))
}

# Update references in Address_df copy for ServiceDepartment
A_copy['identifier'] = A_copy['identifier'].map(sd_address_mapping).fillna(A_copy['identifier'])

# Update references in ContactPoint_df copy for ServiceDepartment
CP_copy['identifier'] = CP_copy['identifier'].map(sd_contact_mapping).fillna(CP_copy['identifier'])

# Create mapping for Personnel
hp_uuid_mapping = {
    old_valuesHP['identifier'].iloc[i]: HP_copy['identifier'].iloc[i]
    for i in range(len(HP_copy))
}

# Update references in Person_df copy
P_copy['identifier'] = P_copy['identifier'].map(hp_uuid_mapping).fillna(P_copy['identifier'])

In [9]:
old_valuesHO

{'identifier': 0    3e70f16a-5548-4822-9e1b-372ad3fbf47a
 1    b306d1a8-e5ee-4c76-948b-2758d7ab7928
 2    9b38fe80-3042-4325-a28f-5ab01fdb8b32
 3    e786ab37-5bca-47be-8298-17c53308fb2e
 4    7b59051b-f400-48d7-831d-5a973d792fa1
 5    d675ebf7-4fe3-4c9a-9371-0f577e9cf84f
 6    ae55cdff-34ab-48fd-8a68-e88e0ad40415
 7    bb0378eb-7a62-422e-9d69-d9fc4b1cb8bd
 8    8e751eb7-64d0-4913-991b-8adf0202861c
 9    26bdd974-d3b5-44b0-8be0-4c3e5c949381
 Name: identifier, dtype: object,
 'address': 0    d4713d60-c8a7-4639-ab11-67b367a9c378
 1    247a8333-f7b0-47d2-8da8-056c3d15eef7
 2    adf20806-e521-4606-b717-6e84d977e993
 3    31d0b664-0589-4877-9b02-52440950fd13
 4    d86dbf11-2880-4c5d-ad1b-8f60c9e4dab2
 5    b732d46f-21e1-4094-9efe-e464da90f534
 6    fa83ada4-a212-4ac5-b689-a4a5ffda0336
 7    d8ab0b30-0ac0-4f0d-9974-c146e8ec01b3
 8    673617d9-4d7b-4307-9224-11e6ba8982dd
 9    6c4c3935-379d-4da1-ade6-c5e9b6e355f6
 Name: address, dtype: object,
 'contactPoint': 0    23a7711a-8133-4876-b7eb-dcd9

In [10]:
import os

# Create the new directory if it doesn't exist
if not os.path.exists('testdata_dup'):
    os.makedirs('testdata_dup')

# Save each dataframe to CSV in the new directory
A_copy.to_csv('testdata_dup/address_d.csv', index=False)
CP_copy.to_csv('testdata_dup/ContactPoint_d.csv', index=False)
HO_copy.to_csv('testdata_dup/HealthcareOrganization_d.csv', index=False)
HP_copy.to_csv('testdata_dup/HealthcarePersonnel_d.csv', index=False)
P_copy.to_csv('testdata_dup/Person_d.csv', index=False)
SD_copy.to_csv('testdata_dup/ServiceDepartment_d.csv', index=False)

In [ ]:
# Update golden_record with new UUIDs for duplicate_id

# Create a combined mapping of old identifiers to new UUIDs
uuid_mapping = {}
uuid_mapping.update(HO_uuid_mapping)  # Healthcare Organizations
uuid_mapping.update(sd_uuid_mapping)  # Service Departments
uuid_mapping.update(hp_uuid_mapping)  # Healthcare Personnel

# Also add mappings for addresses and contact points
uuid_mapping.update(ho_address_mapping)  # Healthcare Org addresses
uuid_mapping.update(ho_contact_mapping)  # Healthcare Org contact points
uuid_mapping.update(sd_address_mapping)  # Service Dept addresses
uuid_mapping.update(sd_contact_mapping)  # Service Dept contact points

# For duplicate_ids that are in the format original_id + '_var1', '_var2', etc.
# Create mappings for them
pattern_mapping = {}
for original_id, new_id in uuid_mapping.items():
    # Create mappings for variations like original_id_var1, original_id_var2, etc.
    for i in range(1, 6):  # Assuming variations up to _var5
        old_var_id = f"{original_id}_var{i}"
        new_var_id = fake.uuid4()  # Generate a completely new UUID
        pattern_mapping[old_var_id] = new_var_id

# Make a copy of the golden_record
updated_golden_record = golden_record.copy()

# Update the duplicate_id column
# First, update using direct mappings
updated_golden_record['duplicate_id'] = updated_golden_record['duplicate_id'].map(uuid_mapping).fillna(updated_golden_record['duplicate_id'])

# Then, update using pattern-based mappings
updated_golden_record['duplicate_id'] = updated_golden_record['duplicate_id'].map(pattern_mapping).fillna(updated_golden_record['duplicate_id'])

# Update the original_id column using direct mappings
updated_golden_record['original_id'] = updated_golden_record['original_id'].map(uuid_mapping).fillna(updated_golden_record['original_id'])

# Preview the results
print(f"Original golden record shape: {golden_record.shape}")
print(f"Updated golden record shape: {updated_golden_record.shape}")
print("\nSample of updated golden record:")
updated_golden_record.head()

In [ ]:
# Check how many duplicate_ids were successfully updated
updated_count = sum(updated_golden_record['duplicate_id'] != golden_record['duplicate_id'])
total_count = len(golden_record)

print(f"Updated {updated_count} out of {total_count} duplicate_ids ({updated_count/total_count:.2%})")

# Check if there are any duplicate_ids that still have the _var pattern
var_pattern = updated_golden_record['duplicate_id'].str.contains('_var', na=False)
remaining_var = sum(var_pattern)

print(f"Remaining IDs with '_var' pattern: {remaining_var}")

if remaining_var > 0:
    print("\nSample of records with remaining '_var' pattern:")
    updated_golden_record[var_pattern].head()

In [ ]:
# Save the updated golden record
updated_golden_record.to_csv('testdata_dup/updated_golden_standard_duplicates.csv', index=False)
print("Updated golden record saved to 'testdata_dup/updated_golden_standard_duplicates.csv'")

In [ ]:
pip install deep-translator